## CNN
Se utilizan directamente las carpetas de imágenes

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
import os
import pickle as pck
import h5py as h5 # para guardar toda el modelo
import numpy as np
import pandas as pd

from skimage import feature, exposure, transform, io

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPool2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score

from keras import backend as K
import tensorflow as tf

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [6]:
estilos = ['abstract-art', 'realism']

In [31]:
path_data = 'data/'
path_train = 'imagenes_color/train/'
path_test = 'imagenes_color/test/'
path_validar = 'imagenes_color/validar/'
path_features = 'features/'
path_training = 'training/'

In [8]:
img_width, img_height = 64, 64

In [9]:
def crear_modelo(num_estilos = 2):
    
    if K.image_data_format() == 'channels_first': 
        input_shape = (3, img_width, img_height) 
    else: 
        input_shape = (img_width, img_height, 3)     
    
    model = Sequential()
    model.add(Conv2D(32, 
                     kernel_size = 5, 
                     input_shape = input_shape, 
                     activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(48, 
                     kernel_size = 3, 
                     activation = 'relu'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.35))
    model.add(Dense(64, activation = 'sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(num_estilos, activation = 'softmax'))

    #optimizer=keras.optimizers.RMSprop()
    model.compile(loss = keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])    
    
    return model
    

In [11]:
modelo = crear_modelo()
modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 60, 60, 32)        2432      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 48)        13872     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 48)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9408)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 9408)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                602176    
__________

In [13]:
def lista_imagenes(path):
    imagenes = []
    for e in estilos:
        archivos = os.listdir(os.path.join(path, e))
        for archivo in archivos:
            imagenes += [(os.path.join(e, archivo), e)]
            
    return imagenes

In [14]:
images_train = lista_imagenes(path_train)
images_test = lista_imagenes(path_test)

In [15]:
print('Imágenes para train: {} \nImágenes para test: {}'.format(len(images_train), len(images_test)))

Imágenes para train: 1198 
Imágenes para test: 238


In [20]:
train_samples = len(images_train) 
test_samples = len(images_test)
batch_size = 16
epochs = 10

In [21]:
train_datagen = ImageDataGenerator(
    rescale = 1. / 255,
    shear_range = 0.2,
    zoom_range = 0.2,   
    horizontal_flip = False)

In [25]:
train_generator = train_datagen.flow_from_directory(path_train, 
                                                    target_size =(img_width, img_height), 
                                                    batch_size = batch_size, 
                                                    color_mode = 'rgb',
                                                    shuffle = True, 
                                                    class_mode = 'categorical')

Found 1198 images belonging to 2 classes.


In [26]:
test_datagen = ImageDataGenerator(rescale = 1. / 255) 

In [27]:
test_generator = test_datagen.flow_from_directory(path_test, 
                                                  target_size =(img_width, img_height), 
                                                  batch_size = batch_size, 
                                                  color_mode = 'rgb',
                                                  shuffle = True, 
                                                  class_mode = 'categorical')

Found 238 images belonging to 2 classes.


In [33]:
modelo.fit_generator(train_generator, 
                     steps_per_epoch = train_samples // batch_size, 
                     epochs = epochs, 
                     validation_data = test_generator,
                     validation_steps = test_samples // batch_size) 
  
modelo.save(path_training + 'model_saved-C.h5') 

Epoch 1/10
74/74 [==============================] - 15s 197ms/step - loss: 0.1367 - acc: 0.9508 - val_loss: 0.4281 - val_acc: 0.8739
Epoch 2/10
74/74 [==============================] - 14s 185ms/step - loss: 0.1156 - acc: 0.9552 - val_loss: 0.4282 - val_acc: 0.8964
Epoch 3/10
74/74 [==============================] - 15s 199ms/step - loss: 0.0961 - acc: 0.9611 - val_loss: 0.6623 - val_acc: 0.8378
Epoch 4/10
74/74 [==============================] - 15s 206ms/step - loss: 0.1151 - acc: 0.9653 - val_loss: 0.6211 - val_acc: 0.8527
Epoch 5/10
74/74 [==============================] - 15s 198ms/step - loss: 0.1100 - acc: 0.9578 - val_loss: 0.4911 - val_acc: 0.8829
Epoch 6/10
74/74 [==============================] - 15s 199ms/step - loss: 0.0861 - acc: 0.9671 - val_loss: 0.5216 - val_acc: 0.8694
Epoch 7/10
74/74 [==============================] - 16s 217ms/step - loss: 0.0944 - acc: 0.9628 - val_loss: 0.5416 - val_acc: 0.8739
Epoch 8/10
74/74 [==============================] - 14s 188ms/step - 

### Se evalua el modelo

In [35]:
modelo.evaluate_generator(generator = test_generator, 
                          steps = test_samples // batch_size)

[0.5496514011342246, 0.8828828818089253]

### Se realiza una prueba

In [60]:
image_path = 'imagenes_color/validar/abstract-art/(abstract-art)481-caeruluus-c8.jpg'

image = load_img(image_path, target_size=(img_width, img_height))  
image = img_to_array(image)  
  
image = image / 255  

image = np.expand_dims(image, axis=0)

In [61]:
prediccion = modelo.predict(image)  

In [62]:
prediccion

array([[0.9867495 , 0.01325054]], dtype=float32)

In [63]:
predict = modelo.predict_classes(image, batch_size = 16)

In [64]:
print(estilos[predict[0]])
predict

abstract-art


array([0])

In [65]:
valida_datagen = ImageDataGenerator(rescale = 1. / 255)
valida_data_generator = valida_datagen.flow_from_directory(path_validar, 
                                                         target_size =(img_width, img_height), 
                                                        batch_size = batch_size, 
                                                        color_mode = 'rgb',
                                                        shuffle = False, 
                                                        class_mode = 'categorical')

test_steps_per_epoch = np.math.ceil(valida_data_generator.samples / valida_data_generator.batch_size)
predictions = modelo.predict_generator(valida_data_generator, steps=test_steps_per_epoch)
predicted_classes = np.argmax(predictions, axis=1)

Found 480 images belonging to 2 classes.


In [66]:
true_classes = valida_data_generator.classes
class_labels = list(valida_data_generator.class_indices.keys())  

In [67]:
confusion = confusion_matrix(true_classes, predicted_classes)
print(confusion)

[[214  26]
 [ 68 172]]


In [68]:
import sklearn.metrics as metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    

              precision    recall  f1-score   support

abstract-art       0.76      0.89      0.82       240
     realism       0.87      0.72      0.79       240

   micro avg       0.80      0.80      0.80       480
   macro avg       0.81      0.80      0.80       480
weighted avg       0.81      0.80      0.80       480

